In [1]:

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt

from sympy.solvers import solve
from sympy import Symbol
import math
import random
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
from numpy import linalg as LA

import time

In [2]:
def function(x,E,theta,eta,w,delta,alpha,s_0,b,rho):

    lambda_ = alpha*(x[4]+s_0)**b
    
    
    A = x[2]*x[3]/w
    mu = 25
    
    #dot_W = x[0]*rho*((mu/x[2]-1)+float(np.random.normal(0,0.22,1)))
    dot_W = x[0]*rho*((mu/x[2]-1))
    #dot_W = 0
    dot_L = theta*(lambda_*(A-x[1])-A)
    
    #dot_L = theta*(lambda_*(E*lambda_-x[1])-E*lambda_ )
    
    dot_p = (x[2]* (1-x[3]) * dot_W / x[0] + eta*(w-x[0])*(E-A+x[1])+ w*dot_L ) / (1-x[3]*w-(1-x[3])*x[0])
    #dot_p = ( eta*(w-x[0])*(E-A+x[1])+ w*dot_L ) 
    
    
    # / (1-x[3]*w-(1-x[3])*x[0])
    dot_n = x[3]* (-(1-w)*dot_p/x[2] + 1/A*(eta*(E-A+x[1])+dot_L))
    #dot_n = 0
    
    #dot_s = -delta*(x[4]) + delta * (np.log(1-dot_p/x[2]))**2
    #dot_s = -delta*(x[4]) + delta * (np.log(1-dot_p/x[2]))**2
    dot_s = -delta*(x[4]) + delta * (-dot_p/x[2])**2
    
    
    return([dot_W, dot_L, dot_p, dot_n, dot_s])
   

In [3]:
def sys(iv, dt, time,function,E,theta,eta,w,delta,alpha,s_0,b,rho):
    n = int(time/dt)
    t = np.linspace(0,time,n)
    x = np.array([[ i for i in iv]]*n)
    n_ord = len(iv)
    
    a = iv
    
    for i in range(1,n):
        X = function(a,E,theta,eta,w,delta,alpha,s_0,b,rho)
        X[0] = X[0] 
        a = a + np.array(X)*dt
        x[i] = a
    return np.array(x)


In [4]:
rho= 0.1
#rho = 100
E = 2
mu = 25
theta = 10
eta = 10
w = 0.5

delta = 2
alpha = 0.01
#alpha = 0.1

s_0 = 10**-6
b = -0.5

In [5]:
T = 300
dt = 10**-2
delta = 0.5

start_time = time.time()

p0 = 29
L0 = 10
x= sys([0.5,L0,p0,0.01,0.01], dt,T,function,E,theta,eta,w,delta,alpha,s_0,b,rho)

print("--- %s seconds ---" % (time.time() - start_time))


n = int(T/dt)
t = np.linspace(0,T,n)

--- 0.3515195846557617 seconds ---


In [8]:
fig = plt.figure(figsize=(5,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(5,1,1)
ax2 = fig.add_subplot(5,1,2)
ax3 = fig.add_subplot(5,1,3)
ax4 = fig.add_subplot(5,1,4)
ax5 = fig.add_subplot(5,1,5)

n = int(T/dt)
t = np.linspace(0,T,n)

#lambda_ = alpha * (x[:,4] + s_0)**b
w = 0.5
#A = x[:,2]*x[:,3]/w

T1 = 4500
T2 = 20000
#xx= np.array([x[500*i] for i in range(int(len(x)//1000))])
#t = np.array([t[500*i] for i in range(int(len(t)//1000))])
xx = x
lambda_ = alpha * (xx[:,4] + s_0)**b

tau = 0.8
#ax1.plot(t,10*(A-x[:,1])/x[:,2], 'r-',label ='W')
ax1.plot(tau*t,xx[:,0], 'k-',label ='W')
ax1.set_ylabel(r'$W$')


ax2.plot(tau*t,xx[:,1], 'k-',label ='L')
#ax2.plot(t,x[:,1]-(1-1/lambda_)*x[:,2]*x[:,3]/w, 'c-',label ='L')
#ax2.plot(t,x[:,1]-(lambda_-1)*E, 'b-',label ='L')
ax2.set_ylabel(r'$L$')


ax3.plot(tau*t,xx[:,2], 'k-',label ='p')
# num = (1-n)*rho*mu+eta*(w-W)*(E+L)-w*theta*lambda_*L
# denom = (1-n)*rho + eta*(w-W)*n/w -theta*n*(lambda_-1)
# ax3.plot(t,num/denom,'k--',alpha=0.5)
ax3.set_ylabel(r'$p$')


ax4.plot(tau*t,xx[:,3], 'k-',label ='n')
#ax4.plot(t,w/x[:,2] * (E+x[:,1]), 'r--',alpha=0.4)
ax4.set_ylabel(r'$n$')


#ax5.plot(t,A/E, 'k--',label ='s',alpha=0.5)
ax5.plot(tau*t,lambda_, 'k-',label ='s')

#ax5.plot(t,(np.log(1-theta*lambda_*(E*(lambda_-1)-x[:,1]) / (x[:,1]+10)))**2, 'g--',label ='s')
ax5.set_ylabel(r'$ \lambda $')
ax5.set_xlabel(r'$ t $')


plt.grid()
plt.show()


## Eigenvalues

In [7]:
def eigenvalue_dis(value):
    tau = dt
    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.075,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((5,5))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    
    Mat[0,0] = rho*(mu/p-1)
    Mat[0,2] = -W*rho*mu/p**2

    Mat[1,1] = -theta*lambda_
    Mat[1,2] = theta*(lambda_-1)*n/w
    Mat[1,3] = theta*(lambda_-1)*p/w
    Mat[1,4] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*Mat[1,2] ) / (1-w*n-(1-n)*W)
    df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*Mat[1,3] ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu/p-1)*eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[2,0] = df3dW
    Mat[2,1] = df3dl
    Mat[2,2] = df3dp
    Mat[2,3] = df3dn
    Mat[2,4] = df3dlambda

    
    df4dl = w/p*(eta+Mat[1,1])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[1,2])
    df4dn = w/p*(-eta*p/w+Mat[1,3])
    df4dlambda = w/(n*p)*theta*(A-L)

    Mat[3,0] = -(1-w)*n/p * df3dW
    Mat[3,1] = -(1-w)*n/p * df3dl + df4dl
    Mat[3,2] = -(1-w)*n * (df3dp/p-dot_p/p**2) + df4dp
    Mat[3,3] = -(1-w)/p *(dot_p + n*df3dn) + df4dn
    Mat[3,4] = -(1-w)*n/p * df3dlambda + df4dlambda


    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    Mat = tau * Mat
    for i in range(5):
        Mat[i,i] = 1 + Mat[i,i]
    eigva, v = LA.eig(Mat)
    return(eigva,v)

In [8]:
def eigenvalue(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.075,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((5,5))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    
    Mat[0,0] = 0
    Mat[0,2] = -rho*mu/p

    Mat[1,1] = -theta*lambda_
    Mat[1,2] = theta*(lambda_-1)*n/w*p
    Mat[1,3] = theta*(lambda_-1)*p/w*n
    Mat[1,4] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)+eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    #df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) - (1-n)*dot_p / (1-n*w-(1-n)*W)**2
    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)**2 # change sign
    df3dl = ( (w-W)*eta + w*Mat[1,1] ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho*mu/p)-(w-W)*eta*(E-L)/p + w*theta*lambda_*L/p ) / (1-w*n-(1-n)*W)
    df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*Mat[1,3] ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu-p)+eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[2,0] = df3dW*w/p
    Mat[2,1] = df3dl/p
    Mat[2,2] = df3dp
    Mat[2,3] = df3dn*n/p
    Mat[2,4] = df3dlambda/p

    
    df4dl = w/p*(eta+Mat[1,1])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[1,2])
    df4dn = w/p*(-eta*p/w+Mat[1,3])
    df4dlambda = w/p*theta*(A-L)

    Mat[3,0] = -(1-w)*n/p * df3dW
    Mat[3,1] = -(1-w)*n/p * df3dl + df4dl
    Mat[3,2] = -(1-w)*n * (df3dp/p-dot_p/p**2) + df4dp
    Mat[3,3] = -(1-w)/p *(dot_p + n*df3dn) + df4dn
    Mat[3,4] = -(1-w)*n/p * df3dlambda + df4dlambda


    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))



    #Mat[4,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dW /p
    #Mat[4,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    #Mat[4,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    #Mat[4,3] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    #Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    Mat[4,0] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dW /p 
    Mat[4,1] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * df3dl /p
    Mat[4,2] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[4,3] = - delta / (2*alpha**2) * tv**2 *lambda_**3*2*dot_p/p * (df3dn /p)
    Mat[4,4] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 - tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    
    eigva, v = LA.eig(Mat)
    return(eigva,v)

In [9]:
list_v = np.copy(x)

In [10]:
def resort(x):
    list_v = np.array([eigenvalue(value)[0] for value in x])
    for i in range(len(x)):
        order = [0,0,0,0,0]
        eig = eigenvalue(x[i])
        order[0],order[1] = eig[0].imag.argsort()[0],eig[0].imag.argsort()[-1]
        order[2],order[3] = eig[0].real.argsort()[0],eig[0].real.argsort()[1]
        order[4] = list(set(([0,1,2,3,4])) - set((order[0:4])))[0]
        order = np.array(order)
        list_v[i] = eig[0][order]
    return(list_v)

## Summer modif 

In [11]:
#list_values = resort(x)
list_values = np.array([eigenvalue(value)[0] for value in x])

In [12]:
T1 = 0
T2 = 20000

In [20]:
plt.figure(figsize=(41,41))
for i in range(5):
    plt.plot(t[T1:T2],list_values[:,i][T1:T2],'.',markersize=0.1)
plt.grid()

/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, order=order)
/home/lcbob/anaconda3/lib/python3.7/site-packages/numpy/core/_asarray.py:85: ComplexWarning: Casting complex values to real discards the imaginary part
  return array(a, dtype, copy=False, ord

In [67]:
eig_vects = eigenvalue(x[5550])[1]

for i in range(len(eig_vects)):
    print(eig_vects[:,i])

[-2.05248015e-04-2.98229966e-05j -6.65908140e-01+1.31461889e-02j
 -7.45777429e-01+0.00000000e+00j -1.41502362e-02+3.04519225e-03j
 -9.09234058e-05+8.82161632e-07j]
[-2.05248015e-04+2.98229966e-05j -6.65908140e-01-1.31461889e-02j
 -7.45777429e-01-0.00000000e+00j -1.41502362e-02-3.04519225e-03j
 -9.09234058e-05-8.82161632e-07j]
[ 6.08501952e-03+0.j -2.85237863e-02+0.j  9.99573875e-01+0.j
 -1.19796215e-03+0.j  3.78905623e-06+0.j]
[-0.01019267+0.j -0.0229152 +0.j  0.99968317+0.j -0.00107591+0.j
  0.0018467 +0.j]
[ 0.01042444+0.j  0.48810341+0.j -0.83378287+0.j  0.01178668+0.j
  0.25751425+0.j]


In [66]:
eigenvalue(x[5550])[0]

array([-10.02473853+1.45860207j, -10.02473853-1.45860207j,
        -0.44974333+0.j        ,   0.29034474+0.j        ,
         0.23929766+0.j        ])

In [63]:
T1 = 0
T2 = 20000
lambda_ = alpha*(x[:,4]+s_0)**b
print(len(x),T1,T2,len(t),len(lambda_))


30000 0 20000 30000 30000


In [9]:
fig = plt.figure(figsize=(8,8))


for i in range(5):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.1)
        
    ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'+--',markersize=1,alpha=0.5)
    #ax2.plot(tt[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],list_values[:,i][T1:T2].imag,'.',markersize=0.5)

ax1.set_ylabel('$\lambda$')        
ax2.set_ylabel('$Re(\lambda_i)$')        
ax3.set_ylabel('$Imag(\lambda_i)$')         
ax3.set_xlabel('$t$')     

plt.show()

0


NameError: name 'list_values' is not defined

In [14]:
len(x)

30000

In [15]:
w,v = eigenvalue(x[int(14000)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-11.105960400037466+11.98555558844652j) 
 [-7.88765710e-05-8.50625395e-05j -6.96186134e-01+1.14389987e-01j
 -7.07545208e-01+0.00000000e+00j -1.26562519e-02+3.81003335e-02j
 -2.78320003e-03-2.04560521e-05j]
(-11.105960400037466-11.98555558844652j) 
 [-7.88765710e-05+8.50625395e-05j -6.96186134e-01-1.14389987e-01j
 -7.07545208e-01-0.00000000e+00j -1.26562519e-02-3.81003335e-02j
 -2.78320003e-03+2.04560521e-05j]
(1.6747941916300277+0j) 
 [ 0.00118529+0.j  0.67539118+0.j -0.73721697+0.j -0.01870966+0.j
 -0.00253645+0.j]
(-0.011201969154594836+0j) 
 [ 0.05528734+0.j -0.91655178+0.j  0.39561769+0.j  0.01888229+0.j
  0.00250141+0.j]
(0.4485088514210197+0j) 
 [ 6.04282043e-03+0.j  1.14977672e-01+0.j -9.93346509e-01+0.j
 -2.48600754e-03+0.j -3.90299383e-04+0.j]


In [16]:

w,v = eigenvalue(x[int(8470)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(-10.63560528550816+9.713925213913676j) 
 [1.06089429e-04+9.67861403e-05j 6.80706159e-01-7.71221026e-02j
 7.27944945e-01+0.00000000e+00j 1.41891713e-02-2.41896336e-02j
 9.86437545e-04+2.62646470e-06j]
(-10.63560528550816-9.713925213913676j) 
 [1.06089429e-04-9.67861403e-05j 6.80706159e-01+7.71221026e-02j
 7.27944945e-01-0.00000000e+00j 1.41891713e-02+2.41896336e-02j
 9.86437545e-04-2.62646470e-06j]
(0.8854889394359972+0j) 
 [ 0.00268883+0.j  0.5636217 +0.j -0.82598877+0.j -0.00805383+0.j
 -0.0010144 +0.j]
(-0.03147441310082165+0j) 
 [ 0.04407726+0.j -0.73680963+0.j  0.67461036+0.j  0.00827107+0.j
  0.00110566+0.j]
(0.3106665542067581+0j) 
 [ 9.47311086e-03+0.j  1.00132812e-01+0.j -9.94928286e-01+0.j
 -1.16262508e-03+0.j -1.83683495e-04+0.j]


In [17]:

w,v = eigenvalue(x[int(8000)])
for i in range(len(w)):
    print(w[i],'\n',v[:,i])

(204452.25041138718+0j) 
 [ 1.02204046e-12+0.j -9.87323560e-05+0.j -1.09409617e-04+0.j
 -1.87275433e-05+0.j -9.99999989e-01+0.j]
(-1.1085592328343823+0j) 
 [ 0.00126559+0.j -0.68332955+0.j  0.72984894+0.j -0.01589982+0.j
  0.01126263+0.j]
(1.4752754376297113+0j) 
 [-0.00094678+0.j -0.67793241+0.j  0.73488896+0.j -0.01597839+0.j
  0.0094693 +0.j]
(-22.911334526950462+51.07513631936653j) 
 [-1.03845421e-05-2.31569991e-05j -6.66639543e-01+9.13617140e-03j
 -7.43817100e-01+0.00000000e+00j -8.88572229e-03-3.19649293e-03j
 -2.74662975e-02+3.74272652e-02j]
(-22.911334526950462-51.07513631936653j) 
 [-1.03845421e-05+2.31569991e-05j -6.66639543e-01-9.13617140e-03j
 -7.43817100e-01-0.00000000e+00j -8.88572229e-03+3.19649293e-03j
 -2.74662975e-02-3.74272652e-02j]


In [247]:
print(669/0.121 ,-255.9/-0.245 )

5528.925619834711 1044.4897959183675


## Jacobian 4x4 without W

In [86]:
def eigenvalue(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.075,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((4,4))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    

    Mat[0,0] = -theta*lambda_
    Mat[0,1] = theta*(lambda_-1)*n/w
    Mat[0,2] = theta*(lambda_-1)*p/w
    Mat[0,3] = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    df3dW = 0
    df3dl = ( w*Mat[0,0] ) / (1-w*n-(1-n)*W)
    df3dp =  ( w*Mat[0,1] ) / (1-w*n-(1-n)*W)
    df3dn =  ( w*Mat[0,2] ) / (1-w*n-(1-n)*W) 
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)

    Mat[1,0] = df3dl
    Mat[1,1] = df3dp
    Mat[1,2] = df3dn
    Mat[1,3] = df3dlambda

    
    df4dl = w/p*(eta+Mat[0,0])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+Mat[0,1])
    df4dn = w/p*(-eta*p/w+Mat[0,2])
    df4dlambda = w/(n*p)*theta*(A-L)

    Mat[2,0] = -(1-w)*n/p * df3dl + df4dl
    Mat[2,1] = -(1-w)*n * (df3dp/p-dot_p/p**2) + df4dp
    Mat[2,2] = -(1-w)/p *(dot_p + n*df3dn) + df4dn
    Mat[2,3] = -(1-w)*n/p * df3dlambda + df4dlambda


    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))

    Mat[3,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * df3dl /p
    Mat[3,1] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dp /p - dot_p/p**2)
    Mat[3,2] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    Mat[3,3] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    eigva, v = LA.eig(Mat)
    return(eigva,v,Mat)

In [73]:
list_values = np.array([eigenvalue(value)[0] for value in x])

In [75]:
fig = plt.figure(figsize=(8,8))


for i in range(5):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    ax1.plot(t[T1:T2],lambda_[T1:T2], 'g-',label ='s',markersize=0.2)
        
    ax2.plot(t[T1:T2],dt*list_values[:,i][T1:T2].real,'b.',markersize=0.4,alpha=0.5)
    ax2.plot(t[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    ax3.plot(t[T1:T2],dt*list_values[:,i][T1:T2].imag,'b.',markersize=0.4)

         
         
plt.show()

0
1
2
3
4


C:\Users\kk\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys
C:\Users\kk\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\kk\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a

IndexError: index 4 is out of bounds for axis 1 with size 4

In [152]:
eigenvalue(x[int(10000)])

(array([-243.62709139,  -66.41088948,   -0.24614738,   -0.94931509,
          -2.44546076]),
 array([[ 4.53878704e-07,  1.72212858e-05, -5.05837076e-03,
         -1.37005536e-03, -5.84391751e-04],
        [ 3.62810441e-02,  2.60539655e-01,  8.59330920e-01,
          8.33303161e-01, -7.91333535e-01],
        [ 4.75118968e-02,  4.91347250e-01, -5.10649350e-01,
         -5.52264762e-01, -6.11260945e-01],
        [ 4.85395502e-03, -1.24173118e-02,  2.31350011e-02,
          2.27590569e-02, -1.08526390e-02],
        [-9.98199752e-01,  8.30989036e-01,  1.50597318e-02,
          9.46695867e-03, -5.75963251e-03]]))

In [238]:
w,v,Mat=eigenvalue(x[int(8460)])
print(w,'\n',v)

ValueError: not enough values to unpack (expected 3, got 2)

In [88]:
v[:,0]

array([ 0.03209412,  0.03209412,  0.01069415, -0.99891219])

In [89]:
Mat

array([[-4.92629410e+01,  2.50493848e+01,  1.61377524e+03,
         2.00951974e+01],
       [-4.92629410e+01,  2.50493848e+01,  1.61377524e+03,
         2.00951974e+01],
       [-5.72926967e-01,  2.85821932e-01,  1.75196695e+01,
         1.37670593e+00],
       [ 1.61630472e+02, -8.73140469e+01, -5.29475604e+03,
        -1.66233292e+02]])

In [94]:
np.dot(Mat,v[:,0])/v[:,0]

array([-111.93645121, -111.93645121, -111.93645121, -111.93645121])

In [95]:
v[:,0]

array([ 0.03209412,  0.03209412,  0.01069415, -0.99891219])

### Jacobian fast

array([[ 0.03209412,  0.20976333,  0.76472271,  0.7070281 ],
       [ 0.03209412,  0.20976333, -0.64267921,  0.7070281 ],
       [ 0.01069415, -0.01640427,  0.03291101,  0.00956888],
       [-0.99891219,  0.95484532,  0.03285526,  0.01144395]])

In [44]:
def eigenvalue_fast(value):

    #parameter
    theta,eta,s0,alpha,bb,mu,rho,w,tv = [10,10,0,0.075,-0.5,25,0.1,0.5,0.1]

    #value
    W,L,p,n,s = value
    
    Mat = np.zeros((3,3))
    A = n*p / w
    lambda_ = alpha*(s+s0)**bb
    
#     Mat[0,0] = rho*(mu/p-1)
#     Mat[0,2] = -W*rho*mu/p**2

    df2dL = -theta*lambda_
    df2dp = theta*(lambda_-1)*n/w
    df2dn = theta*(lambda_-1)*p/w
    df2dlambda = theta*(A-L)

    dot_L = theta*(lambda_*(A-L)-A)
    dot_p = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*dot_L) / (1-n*w-(1-n)*W)

    df3dW = -eta*(E-A+L) / (1-w*n-(1-n)*W) + (1-n)*dot_p / (1-n*w-(1-n)*W)
    df3dl = ( (w-W)*eta + w*df2dL ) / (1-w*n-(1-n)*W)
    df3dp =  ( (1-n)*(-rho)-(w-W)*eta*n/w + w*df2dp ) / (1-w*n-(1-n)*W)
    df3dn =  ( -rho*(mu-p)-(w-W)*eta*p/w + w*df2dn ) / (1-w*n-(1-n)*W) - (-w+W)*((1-n)*rho*(mu/p-1)*eta*(E-A+L)+w*dot_L) / (1-w*n-(1-n)*W)**2
    df3dlambda = w*(theta*(A-L)) / (1-w*n-(1-n)*W)


    
    df4dl = w/p*(eta+Mat[1,1])
    df4dp = -w/p**2*(eta*(E-A+L)+theta*(lambda_*(A-L)-A)) + w/p * (eta*(-n/w)+df2dp)
    df4dn = w/p*(-eta*p/w+df2dn)
    df4dlambda = w/(n*p)*theta*(A-L)

    Mat[0,0] = -(1-w)/p *(dot_p + n*df3dn) + df4dn
    Mat[0,1] = -(1-w)*n/p * df3dlambda + df4dlambda


    a0 = ((1-n)*rho*(mu-p)*eta*(w-W)*(E-A+L)+w*theta*(-A)) / (p*(1-n*w-(1-n)*W))
    a1 = w*theta*(A-L) / (p*(1-n*w-(1-n)*W))

    Mat[1,0] = delta / (2*alpha**2) * tv**2 *lambda_**2*2*dot_p/p * (df3dn /p)
    Mat[1,1] = delta / (2*alpha**2) * (alpha**2 - 3*s0*lambda_**2 + tv**2*lambda_**2*(3*a0**2+2*4*a0*a1*lambda_+5*a1**2*lambda_**2))

    eigva, v = LA.eig(Mat)
    return(eigva,v)

In [42]:
list_values = np.array([eigenvalue_fast(value)[0] for value in x])

In [43]:
fig = plt.figure(figsize=(8,8))


for i in range(5):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    ax1.plot(t[T1:T2],lambda_[T1:T2], 'g.-',label ='s',markersize=0.1)
        
    ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'.-',markersize=2,alpha=0.5)
    ax2.plot(t[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    ax3.plot(t[T1:T2],list_values[:,i][T1:T2].imag,'.',markersize=0.7)

         
         
plt.show()

0
1
2
3


C:\Users\kk\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  import sys
C:\Users\kk\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a future version, a new instance will always be created and returned.  Meanwhile, this warning can be suppressed, and the future behavior ensured, by passing a unique label to each axes instance.
  
C:\Users\kk\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: MatplotlibDeprecationWarning: Adding an axes using the same arguments as a previous axes currently reuses the earlier instance.  In a

IndexError: index 3 is out of bounds for axis 1 with size 3

## Discrete time (continuos eig)

In [155]:
def resort_dis(x):
    list_v = np.array([eigenvalue_dis(value)[0] for value in x])
    for i in range(len(x)):
        order = [0,0,0,0,0]
        eig = eigenvalue_dis(x[i])
        order[0],order[1] = eig[0].imag.argsort()[0],eig[0].imag.argsort()[-1]
        order[2],order[3] = eig[0].real.argsort()[0],eig[0].real.argsort()[1]
        order[4] = list(set(([0,1,2,3,4])) - set((order[0:4])))[0]
        order = np.array(order)
        list_v[i] = eig[0][order]
    return(list_v)

In [157]:
list_values = resort_dis(x)

In [161]:
T1 = 100000
T2 = 200000
print(len(x),T1,T2)

300000 100000 200000


In [162]:
fig = plt.figure(figsize=(8,8))


for i in range(5):
    print(i)
    fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
    ax1 = fig.add_subplot(3,1,1)
    ax2 = fig.add_subplot(3,1,2)
    ax3 = fig.add_subplot(3,1,3)
    
    ax1.plot(t[T1:T2],lambda_[T1:T2], 'g.-',label ='s',markersize=0.1)
        
    ax2.plot(t[T1:T2],list_values[:,i][T1:T2].real,'.-',markersize=2,alpha=0.5)
    ax2.plot(t[T1:T2],0+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],-1+0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    #ax2.plot(t[T1:T2],np.absolute(list_values[:,i][T1:T2]),'-',alpha=0.5)
    
    ax3.plot(t[T1:T2],0*list_values[:,i][T1:T2].real,'k--',alpha=0.5)
    ax3.plot(t[T1:T2],list_values[:,i][T1:T2].imag,'.',markersize=0.7)

         
         
plt.show()

0
1
2
3
4


In [163]:
eigenvalue(x[180000])

(array([-2.36956615e+04,  2.09929709e+02, -4.03598451e-02,  4.31380296e-01,
         2.29254710e+00]),
 array([[ 3.92950545e-12,  3.53833670e-07, -8.26030987e-01,
          1.37734064e-04, -4.95009397e-05],
        [ 8.37763754e-04, -6.90621663e-01, -4.66545185e-01,
         -8.39771897e-01,  2.61230174e-01],
        [ 7.78091244e-04, -6.20841851e-01, -3.12174883e-01,
         -5.42909059e-01,  9.64999244e-01],
        [ 2.60868443e-06, -1.19934391e-03,  1.04744029e-03,
          1.37995566e-03, -1.76451984e-02],
        [-9.99999346e-01,  3.70938911e-01, -5.05385453e-02,
          5.56696479e-03, -1.49632554e-02]]))

In [66]:
eigenvalue(x[3448])

(array([-28.62712303+72.20568869j, -28.62712303-72.20568869j,
         -2.72155594 +0.j        ,  -0.61922709 +0.j        ,
         -0.34114133 +0.j        ]),
 array([[-6.72081739e-06-1.69635008e-05j, -6.72081739e-06+1.69635008e-05j,
         -7.97815405e-05+0.00000000e+00j, -3.29176641e-03+0.00000000e+00j,
         -6.14298383e-03+0.00000000e+00j],
        [-2.94709951e-01-4.79705810e-02j, -2.94709951e-01+4.79705810e-02j,
         -9.93673602e-01+0.00000000e+00j,  3.61473625e-02+0.00000000e+00j,
         -1.76096061e-02+0.00000000e+00j],
        [-7.17650265e-01+0.00000000e+00j, -7.17650265e-01-0.00000000e+00j,
         -1.09159461e-01+0.00000000e+00j, -9.99328205e-01+0.00000000e+00j,
         -9.99820431e-01+0.00000000e+00j],
        [-5.41214732e-03-2.19557792e-02j, -5.41214732e-03+2.19557792e-02j,
         -2.20953477e-02+0.00000000e+00j,  4.49574163e-03+0.00000000e+00j,
          3.17850529e-03+0.00000000e+00j],
        [ 6.20066669e-01+1.04062210e-01j,  6.20066669e-01-1.0406221

In [68]:
eigenvalue(x[3492])

(array([-11.84167295+0.j        ,  -8.47793586+0.j        ,
          3.35413457+2.70068833j,   3.35413457-2.70068833j,
         -0.18530296+0.j        ]),
 array([[-1.46835276e-04+0.j        , -2.19372259e-04+0.j        ,
         -3.52570111e-04+0.00028238j, -3.52570111e-04-0.00028238j,
         -1.22174649e-02+0.j        ],
        [-5.30631145e-01+0.j        , -4.23258223e-01+0.j        ,
         -2.68045284e-01-0.13536369j, -2.68045284e-01+0.13536369j,
          5.48374388e-02+0.j        ],
        [-8.47522965e-01+0.j        , -9.05980578e-01+0.j        ,
          9.52644893e-01+0.j        ,  9.52644893e-01-0.j        ,
         -9.98404505e-01+0.j        ],
        [-7.19439791e-03+0.j        , -1.84392783e-03+0.j        ,
         -2.92569211e-04+0.00489346j, -2.92569211e-04-0.00489346j,
          3.72998408e-03+0.j        ],
        [ 9.14499964e-03+0.j        ,  6.94410740e-03+0.j        ,
          2.18845747e-02+0.04234308j,  2.18845747e-02-0.04234308j,
          4.256792

In [69]:
eigenvalue(x[3430])

(array([-26.11891577+69.73550862j, -26.11891577-69.73550862j,
         -2.82406922 +0.j        ,  -0.59814644 +0.j        ,
         -0.3604201  +0.j        ]),
 array([[ 7.01604509e-06+1.87462380e-05j,  7.01604509e-06-1.87462380e-05j,
         -5.85794345e-05+0.00000000e+00j,  3.45113319e-03+0.00000000e+00j,
          5.85883758e-03+0.00000000e+00j],
        [ 2.97703525e-01+5.03877667e-02j,  2.97703525e-01-5.03877667e-02j,
         -9.96249391e-01+0.00000000e+00j, -2.84756973e-02+0.00000000e+00j,
          1.35019764e-02+0.00000000e+00j],
        [ 7.45895203e-01+0.00000000e+00j,  7.45895203e-01-0.00000000e+00j,
         -8.22245433e-02+0.00000000e+00j,  9.99577351e-01+0.00000000e+00j,
          9.99886047e-01+0.00000000e+00j],
        [ 5.67958701e-03+2.24980878e-02j,  5.67958701e-03-2.24980878e-02j,
         -2.26701939e-02+0.00000000e+00j, -4.22567172e-03+0.00000000e+00j,
         -3.17008339e-03+0.00000000e+00j],
        [-5.87964727e-01-7.89499088e-02j, -5.87964727e-01+7.8949908

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

m=25

A = x[:,2]*x[:,3] / w
lambda_ = alpha * (x[:,4] + s_0)**b


# ax.plot(x[:,1], A,lambda_,"-",markersize=0.5,alpha=0.5)
# ax.plot(x[:,1], x[:,2],lambda_,"c-",markersize=0.5,alpha=0.5)
ax.plot(x[:,3], (A-x[:,1])/x[:,2],lambda_,"r.",markersize=0.5,alpha=0.5)


ax.set_xlabel('a0')
ax.set_ylabel('a1')
ax.set_zlabel('lambda')

plt.show()

In [7]:
len(x[:,2][1:])

19999

### Test _Slow-Fast

In [506]:
#is \dot{p}/pslow ?
fig = plt.figure(figsize=(5,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(5,1,1)
ax2 = fig.add_subplot(5,1,2)
ax3 = fig.add_subplot(5,1,3)
ax4 = fig.add_subplot(5,1,4)
ax5 = fig.add_subplot(5,1,5)


T1,T2 = int(0/dt),int(10/dt)

ax1.plot(t[T1:T2],x[:,1][T1:T2], 'm-')
ax1.plot(t[T1:T2],(1-1/lambda_[T1:T2])*A[T1:T2])

ax2.plot(t[T1:T2],A[T1:T2], 'm-')
ax2.plot(t[T1:T2],E+x[:,1][T1:T2])

ax3.plot(t[T1:T2],x[:,3][T1:T2], 'm-')
ax3.plot(t[T1:T2],E*lambda_[T1:T2]*w/x[:,2][T1:T2])



dotp = (1-E*lambda_[T1:T2]*w/x[:,2][T1:T2])*rho*(mu/x[:,2][T1:T2]-1) / (1-x[:,3][T1:T2]*w-(1-x[:,3][T1:T2])*x[:,0][T1:T2])

#dotp = (eta*(w-x[:,0])*(E+x[:,1]-A)+w*theta*(lambda_*(A-x[:,1])-A)) / (1-x[:,3]*w-(1-x[:,3])*x[:,0])

ax4.plot(t[T1:T2],x[:,4][T1:T2], 'm-')
ax4.plot(t[T1:T2],(dotp[T1:T2]/x[:,2][T1:T2])**2)


#ax4.plot(t[T1:T2],lambda_[T1:T2], 'm-')
#ax4.plot(t[T1:T2],(1-s_0*lambda_[T1:T2]**2/alpha**2+lambda_[T1:T2]**2/alpha**2*(dotp[T1:T2]/x[:,2][T1:T2])**2))


ax5.plot(t[T1:T2],x[:,2][T1:T2], 'm-')
#ax5.plot(t[T1:T2],x[:,4])


ax1.set_ylabel('L')
ax2.set_ylabel('A')

ax3.set_ylabel('n')
ax4.set_ylabel('s')



plt.show()

In [19]:
#is \dot{p}/pslow ?
fig = plt.figure(figsize=(5,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(1,1,1)

ax1.plot(t,1/(1-w*x[:,3]-(1-x[:,3])*x[:,0]), 'm-')
ax1.plot(t,x[:,3])
ax1.plot(t,x[:,0])
plt.show()

In [47]:
#is \dot{p}/pslow ?
fig = plt.figure(figsize=(5,5))
fig.subplots_adjust(wspace = 0.5, hspace = 0.3)
ax1 = fig.add_subplot(1,1,1)

A = x[:,3]*x[:,2]/w

# ax1.plot(t[1:],(1-1/lambda_[1:])*A[1:], 'g--',alpha=0.5)
# ax1.plot(t[1:],x[:,1][1:], 'b-',alpha=0.5)


dot_A= (x[:,3][1:]*x[:,2][1:]-x[:,3][:-1]*x[:,2][:-1])/(w*dt)
dot_lambda = (lambda_[1:]-lambda_[:-1])/dt
ax1.plot(t[1:],(x[:,1][1:]-x[:,1][:-1])/(dt), 'g.',alpha=0.5)
ax1.plot(t[1:],dot_A*(1-1/lambda_[1:])+A[1:]*dot_lambda/lambda_[1:], 'b--',alpha=0.5)
ax1.plot(t[1:],dot_A*(1-1/lambda_[1:]), 'r--',alpha=0.5)
# ax1.plot(t[1:],A[1:]*dot_lambda/lambda_[1:], 'k--',alpha=0.5)


# ax1.plot(t[1:],(x[:,2][1:]-x[:,2][:-1])/(dt*x[:,2][1:]), 'm-')
# ax1.plot(t[1:],(x[:,3][1:]-x[:,3][:-1])/(dt*x[:,3][1:]), 'k-',alpha=0.5)
# ax1.plot(t[1:],(x[:,1][1:]-x[:,1][:-1])/(dt*x[:,1][1:]), 'g-',alpha=0.5)
# ax1.plot(t[1:],lambda_[1:], 'g--',alpha=0.5)

# ax1.plot(t[1:],(x[:,4][1:]-x[:,4][:-1])/(dt*x[:,4][1:]), 'c-',alpha=0.5)
# ax1.plot(t[1:],(x[:,0][1:]-x[:,0][:-1])/(dt*x[:,0][1:]), 'b-',alpha=0.5)

#ax1.plot(t[1:],(x[:,1][1:]-x[:,1][:-1])/(dt*x[:,1][1:]), 'k-',alpha=0.3)



plt.show()

In [182]:
W,L,p,n,s = np.transpose(x)
w=0.5
x_a0 = -(rho*(mu/p-1)*(1-n) + 1/p *eta*(w-W)*(E-n*p/w+L)- theta *n) 
x_a1 = (theta*w*(p*n/w-L)) / (p)

# x_a0 = rho*(mu/p-1)
# x_a1 = -rho*(mu/p-1)*E*p/w


# x_a0 = rho*(mu/p-1)
# x_a1 = -rho*(mu/p-1)*E*w/p

In [183]:
plt.figure()
plt.plot(x_a0)
plt.plot(x_a1)
#plt.plot((-5*lambda_**4*x_a1**2 -4*lambda_**3*2*x_a0*x_a1 -3* lambda_**2*(x_a0**2+s_0) + alpha**2))



plt.show()

## Slow Manifold

In [199]:
l_a0 = np.linspace(-4,4,300)
l_a1 = np.linspace(-1,1,50)
#l_a1 = np.logspace(np.log(0.1)/np.log(10), np.log(120)/np.log(10), 100)
# a1 = 1
# root_real = [np.real([np.roots([-a1**2,-2*a0*a1,-a0**2-s_0,0,alpha**2 ])[i] for a0 in l_a0]) for i in range(4)]
# root_imag = np.imag([np.roots([-a1**2,-2*a0*a1,-a0**2-s_0,0,alpha**2 ]) for a0 in l_a0])


In [200]:
def pos_or_nan(x):
    if x >= 0:
        return(x)
    else:
        return(np.nan)
    
def lim_or_nan(x,c):
    if abs(x) < c:
        return(x)
    else:
        return(np.nan)

In [230]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

a0,a1 = l_a0[0],l_a1[0]

#color = ['b','r','g','k','y']
color = ['b']*5

opacity = [0.2]*5
for a1 in l_a1:
    root_real = [np.real([np.roots([-(a1)**2,2*a0*(a1),-a0**2-s_0,0,alpha**2,0 ])[i] for a0 in l_a0]) for i in range(len(np.roots([-(-a1+a0)**2,2*a0*(-a1+a0),-a0**2-s_0,0,alpha**2,0 ])))]
    for i in range(len(root_real)):
        z = [pos_or_nan(j) for j in root_real[i]]
        ax.plot(l_a0,[a1]*len(l_a0),z,c=color[i],alpha=opacity[i])
        
ax.set_zlim3d(0.5,40)

plt.show()

In [235]:
T1,T2 = int(50/dt),int(150/dt)

W,L,p,n,s = np.transpose(x)
A = n*p /w
# x_a0 = rho*(mu/p-1)
# x_a1 = -rho*(mu/p-1)*E*p/w

x_a0 = - (rho*(mu/p-1)*(1-n) + 1/p *eta*(w-W)*(E-n*p/w+L)- theta *n) 
x_a1 = (theta*w*(A-L)) / (p)

T1, T2 = 5,60
ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'g-',alpha=0.8)
ax.plot([x_a0[T1]],[x_a1[T1]],[lambda_[T1]],'r.',alpha=0.5)

T1,T2 = int(50/dt),int(150/dt)
ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'k-',alpha=0.5)


x_a0 =  theta *n 
x_a1 = (theta*n*(E/(E+L)))
#ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'k.-',alpha=0.5)        
        
        
# x_a0 = (plt.plot(delta*x[:,4])
# ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'k-',alpha=0.8)
# ax.set_zlim3d(-1,20)

ax.set_xlabel('$a_0$')
ax.set_ylabel('$a_1$')
ax.set_zlabel('$\lambda$')

plt.show()


In [210]:

x_a0 =  theta *n
#x_a1 = theta*w*(A-L)/p
x_a1 = theta*(-W/p*(E-A+L)+w*E/p+(1-n)*rho*(mu/p-1))

# x_a0 = (rho*(mu/p-1)*(1-n) + 1/p *eta*(w-W)*(E-n*p/w+L)- theta *n) 
# x_a1 = (theta*w*(p*n/w-L)) / (p)

ax.plot(x_a0[T1:T2],x_a1[T1:T2],lambda_[T1:T2],'k-',alpha=0.5)
plt.show()

In [65]:
plt.figure()
# plt.plot(eta*(w-W)*(E-n*p/w+L))
# plt.plot(L-(1-1/lambda_)*A)
# plt.plot(x[:,4])
# plt.plot(lambda_)
#plt.plot(delta*x[:,4])
#plt.plot( ((rho*(mu/p-1)*(1-n)+1/p * eta*(w-W)*(E-A+L) + theta*w/p*(lambda_*(A-L)-A)) / (1-n*w-(1-n)*W))**2)

dotp = ((rho*(mu/p-1)*(1-n)+1/p * eta*(w-W)*(E-A+L) + theta*w/p*(lambda_*(A-L)-A)) / (1-n*w-(1-n)*W))
plt.plot( ((rho*(mu/p-1)*(1-n)*p+eta*(w-W)*(E-A+L) + theta*w*(lambda_*(A-L)-A)) / (1-n*w-(1-n)*W)))
plt.plot(theta*(lambda_*(A-L)-A))
plt.plot((-(1-w)*dotp/p+1/A*(eta*(E-A+L)+theta*(lambda_*(A-L)-A))))
plt.plot((-delta*(-s+(dotp/p)**2)))
plt.plot(n*dotp+eta*(E-A+L)+theta*(lambda_*(A-L)-A),'k-')
plt.plot(rho*(mu/p-1),'y')
plt.show()

## Stability on the slow Manifold

In [91]:
def derivative(z,x_a0,x_a1):
    lambda_ = np.array(z)
    alpha =0.1
    s_0 = 10**-6
    der = (delta)*(-5*lambda_**4*x_a1**2 -4*lambda_**3*2*x_a0*x_a1 -3* lambda_**2*(x_a0**2+s_0) + alpha**2)
    return(der)
    

In [94]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

a0,a1 = l_a0[0],l_a1[0]
for a1 in l_a1:
    root_real = [np.real([np.roots([-a1**2,-2*a0*a1,-a0**2-s_0,0,alpha**2 ])[i] for a0 in l_a0]) for i in range(len(np.roots([-a1,-2*a0*a1,-a0-s_0,0,alpha ])))]
    for i in range(len(root_real)):
        z = [pos_or_nan(j) for j in root_real[i]]
        y = np.array([a1]*len(l_a0))
        ax.plot(l_a0,y,derivative(z,l_a0,y),'m-',alpha=0.5)
plt.show()

In [95]:

lim_der = 5000

der = (delta)*(-5*lambda_**4*x_a1**2 -4*lambda_**3*2*x_a0*x_a1 -3* lambda_**2*(x_a0**2+s_0) + alpha**2)[T1:T2]
der = [lim_or_nan(x,lim_der) for x in der]
ax.plot(x_a0[T1:T2],x_a1[T1:T2],der,'k-.',alpha=0.8,markersize=0.5,linewidth=0.4)

ax.set_zlim3d(-20,20)

plt.show()

25000